In [ ]:
pip install wandb

## Import necessary libraries

In [1]:
### Necessary Imports and dependencies
import os
import shutil
import time
import math
from enum import Enum
from functools import partial
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision.transforms import v2
import torchvision.transforms as transforms
from typing import Any, Dict, Union, Type, Callable, Optional, List
from torchvision.models.vision_transformer import MLPBlock
import wandb
import json
from PIL import Image
from torch.utils.data import Dataset
import random
import torchvision.transforms.functional as TF
import numpy as np
import torchvision.transforms as transforms
from PIL import ImageOps, ImageEnhance
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cpu')

## Epochs and Batch_size

In [4]:
num_epochs = 90
batch_size = 256

## Data preprocessing and Loading

In [5]:
class RandAugment:
    def __init__(self, n=9, m=0.5):
        self.n = n
        self.m = m  # [0, 30] in paper, but we use [0, 1] for simplicity
        self.augment_list = [
            self.auto_contrast, self.equalize, self.rotate, self.solarize, 
            self.color, self.contrast, self.brightness, self.sharpness,
            self.shear_x, self.shear_y, self.translate_x, self.translate_y,
            self.posterize, self.solarize_add, self.invert, self.identity
        ]

    def __call__(self, img):
        ops = random.choices(self.augment_list, k=self.n)
        for op in ops:
            img = op(img)
        return img

    def auto_contrast(self, img):
        return ImageOps.autocontrast(img)

    def equalize(self, img):
        return ImageOps.equalize(img)

    def rotate(self, img):
        return TF.rotate(img, self.m * 30)

    def solarize(self, img):
        return TF.solarize(img, int((1 - self.m) * 255))

    def color(self, img):
        return TF.adjust_saturation(img, 1 + self.m)

    def contrast(self, img):
        return TF.adjust_contrast(img, 1 + self.m)

    def brightness(self, img):
        return TF.adjust_brightness(img, 1 + self.m)

    def sharpness(self, img):
        return ImageEnhance.Sharpness(img).enhance(1 + self.m)

    def shear_x(self, img):
        return TF.affine(img, 0, [0, 0], 1, [self.m, 0])

    def shear_y(self, img):
        return TF.affine(img, 0, [0, 0], 1, [0, self.m])

    def translate_x(self, img):
        return TF.affine(img, 0, [int(self.m * img.size[0] / 3), 0], 1, [0, 0])

    def translate_y(self, img):
        return TF.affine(img, 0, [0, int(self.m * img.size[1] / 3)], 1, [0, 0])

    def posterize(self, img):
        return TF.posterize(img, int((1 - self.m) * 8))

    def solarize_add(self, img):
        return TF.solarize(TF.adjust_brightness(img, 1 + self.m), int((1 - self.m) * 255))

    def invert(self, img):
        return TF.invert(img) if random.random() < 0.5 else img

    def identity(self, img):
        return img

class Mixup(nn.Module):
    def __init__(self, alpha=0.8):
        super().__init__()
        self.alpha = alpha

    def forward(self, batch):
        images, labels = batch
        lam = np.random.beta(self.alpha, self.alpha)
        batch_size = images.size(0)
        index = torch.randperm(batch_size)
        mixed_images = lam * images + (1 - lam) * images[index, :]
        labels_a, labels_b = labels, labels[index]
        return mixed_images, labels_a, labels_b, lam

class CutMix(nn.Module):
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha

    def forward(self, batch):
        images, labels = batch
        lam = np.random.beta(self.alpha, self.alpha)
        batch_size, _, H, W = images.shape
        cx = np.random.uniform(0, W)
        cy = np.random.uniform(0, H)
        w = W * np.sqrt(1 - lam)
        h = H * np.sqrt(1 - lam)
        x0 = int(np.clip(cx - w // 2, 0, W))
        y0 = int(np.clip(cy - h // 2, 0, H))
        x1 = int(np.clip(cx + w // 2, 0, W))
        y1 = int(np.clip(cy + h // 2, 0, H))
        index = torch.randperm(batch_size)
        images[:, :, y0:y1, x0:x1] = images[index, :, y0:y1, x0:x1]
        lam = 1 - ((x1 - x0) * (y1 - y0) / (W * H))
        labels_a, labels_b = labels, labels[index]
        return images, labels_a, labels_b, lam

class RandomErasing(nn.Module):
    def __init__(self, probability=0.25, sl=0.02, sh=0.4, r1=0.3, r2=1/0.3):
        super().__init__()
        self.probability = probability
        self.sl = sl
        self.sh = sh
        self.r1 = r1
        self.r2 = r2

    def forward(self, img):
        if random.uniform(0, 1) > self.probability:
            return img
        
        for attempt in range(100):
            area = img.size()[1] * img.size()[2]
            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, self.r2)

            h = int(round(np.sqrt(target_area * aspect_ratio)))
            w = int(round(np.sqrt(target_area / aspect_ratio)))

            if w < img.size()[2] and h < img.size()[1]:
                x1 = random.randint(0, img.size()[1] - h)
                y1 = random.randint(0, img.size()[2] - w)
                if img.size()[0] == 3:
                    img[0, x1:x1+h, y1:y1+w] = random.uniform(0, 1)
                    img[1, x1:x1+h, y1:y1+w] = random.uniform(0, 1)
                    img[2, x1:x1+h, y1:y1+w] = random.uniform(0, 1)
                else:
                    img[0, x1:x1+h, y1:y1+w] = random.uniform(0, 1)
                return img
        return img

class LabelSmoothing(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothing, self).__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        n_classes = pred.size(1)
        one_hot = torch.zeros_like(pred).scatter(1, target.unsqueeze(1), 1)
        smooth_one_hot = one_hot * (1 - self.smoothing) + self.smoothing / n_classes
        log_prob = nn.functional.log_softmax(pred, dim=1)
        return torch.mean(torch.sum(-smooth_one_hot * log_prob, dim=1))

# Updated ImageNet100Dataset
class ImageNet100Dataset(torch.utils.data.Dataset):
    def __init__(self, root_dirs, labels_file, transform=None, augment=None, retain=False, forget=False, forget_label="n01818515"):
        self.transform = transform
        self.augment = augment
        self.images = []
        self.labels = []
        self.label_to_idx = {}
        
        with open(labels_file, 'r') as f:
            label_dict = json.load(f)
        
        unique_labels = sorted(label_dict.keys())
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
        
        for root_dir in root_dirs:
            for label in os.listdir(root_dir):
                if retain:
                    if label == forget_label:
                        continue
                    else:
                        label_path = os.path.join(root_dir, label)
                        if os.path.isdir(label_path):
                            for img_name in os.listdir(label_path):
                                img_path = os.path.join(label_path, img_name)
                                self.images.append(img_path)
                                self.labels.append(self.label_to_idx[label])
                elif forget:
                    if label != forget_label:
                        continue
                    else:
                        label_path = os.path.join(root_dir, label)
                        if os.path.isdir(label_path):
                            for img_name in os.listdir(label_path):
                                img_path = os.path.join(label_path, img_name)
                                self.images.append(img_path)
                                self.labels.append(self.label_to_idx[label])
                else:
                    label_path = os.path.join(root_dir, label)
                    if os.path.isdir(label_path):
                        for img_name in os.listdir(label_path):
                            img_path = os.path.join(label_path, img_name)
                            self.images.append(img_path)
                            self.labels.append(self.label_to_idx[label])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        if self.augment:
            image = self.augment(image)
        
        label = torch.tensor(label)
        
        return image, label

# Define transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.05, 1.0)),
    transforms.RandomHorizontalFlip(),
    RandAugment(n=9, m=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    RandomErasing(probability=0.25)
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the datasets
train_dirs = [
    '/kaggle/input/imagenet100/train.X1',
    '/kaggle/input/imagenet100/train.X2',
    '/kaggle/input/imagenet100/train.X3',
    '/kaggle/input/imagenet100/train.X4'
]
val_dir = ['/kaggle/input/imagenet100/val.X']
labels_file = '/kaggle/input/imagenet100/Labels.json'

train_dataset = ImageNet100Dataset(
    root_dirs=train_dirs,
    labels_file=labels_file,
    transform=train_transform,
    retain = True
)

val_dataset = ImageNet100Dataset(
    root_dirs=val_dir,
    labels_file=labels_file,
    transform=val_transform
)

# Custom collate function for Mixup and CutMix
def collate_fn(batch):
    images, labels = torch.utils.data.default_collate(batch)
    if random.random() < 0.5:
        return Mixup(alpha=0.8)((images, labels))
    else:
        return CutMix(alpha=1.0)((images, labels))

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

## Steps

In [6]:
n = len(train_dataset)
total_steps = round((n * num_epochs) / batch_size)
warmup_try = 10000

In [7]:
print(total_steps)

45703


## Model

In [ ]:
class MLPBlock(nn.Module):
    def __init__(self, in_dim, mlp_dim, dropout):
        super().__init__()
        self.linear_1 = nn.Linear(in_dim, mlp_dim)
        self.activation = nn.GELU()
        self.dropout_1 = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(mlp_dim, in_dim)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.activation(x)
        x = self.dropout_1(x)
        x = self.linear_2(x)
        x = self.dropout_2(x)
        return x
    
class EncoderBlock(nn.Module):
    """Transformer encoder block."""

    def __init__(
        self,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float,
        attention_dropout: float,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
    ):
        super().__init__()
        self.num_heads = num_heads

        # Attention block
        self.ln_1 = norm_layer(hidden_dim)
        self.self_attention = nn.MultiheadAttention(hidden_dim, num_heads, dropout=attention_dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

        # MLP block
        self.ln_2 = norm_layer(hidden_dim)
        self.mlp = MLPBlock(hidden_dim, mlp_dim, dropout)

        # Fix init discrepancy between nn.MultiheadAttention and that of big_vision
        bound = math.sqrt(3 / hidden_dim)
        nn.init.uniform_(self.self_attention.in_proj_weight, -bound, bound)
        nn.init.uniform_(self.self_attention.out_proj.weight, -bound, bound)

    def forward(self, input: torch.Tensor):
        torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")
        x = self.ln_1(input)
        x, _ = self.self_attention(x, x, x, need_weights=False)
        x = self.dropout(x)
        x = x + input

        y = self.ln_2(x)
        y = self.mlp(y)
        return x + y


class Encoder(nn.Module):
    """Transformer Model Encoder for sequence to sequence translation."""

    def __init__(
        self,
        num_layers: int,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float,
        attention_dropout: float,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
    ):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        layers: OrderedDict[str, nn.Module] = OrderedDict()
        for i in range(num_layers):
            layers[f"encoder_layer_{i}"] = EncoderBlock(
                num_heads,
                hidden_dim,
                mlp_dim,
                dropout,
                attention_dropout,
                norm_layer,
            )
        self.layers = nn.Sequential(layers)
        self.ln = norm_layer(hidden_dim)

    def forward(self, input: torch.Tensor):
        torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")
        return self.ln(self.layers(self.dropout(input)))


def jax_lecun_normal(layer, fan_in):
    """(re-)initializes layer weight in the same way as jax.nn.initializers.lecun_normal and bias to zero"""

    # constant is stddev of standard normal truncated to (-2, 2)
    std = math.sqrt(1 / fan_in) / .87962566103423978
    nn.init.trunc_normal_(layer.weight, std=std, a=-2 * std, b=2 * std)
    if layer.bias is not None:
        nn.init.zeros_(layer.bias)
        
class SimpleVisionTransformer(nn.Module):
    """Vision Transformer modified per https://arxiv.org/abs/2205.01580."""
    def __init__(
        self,
        image_size: int,
        patch_size: int,
        num_layers: int,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float = 0.0,
        attention_dropout: float = 0.0,
        num_classes: int = 100,
        representation_size: Optional[int] = None,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
    ):
        super().__init__()
        torch._assert(image_size % patch_size == 0, "Input shape indivisible by patch size!")
        self.image_size = image_size
        self.patch_size = patch_size
        self.hidden_dim = hidden_dim
        self.mlp_dim = mlp_dim
        self.attention_dropout = attention_dropout
        self.dropout = dropout
        self.num_classes = num_classes
        self.representation_size = representation_size
        self.norm_layer = norm_layer

        self.conv_proj = nn.Conv2d(
            in_channels=3, out_channels=hidden_dim, kernel_size=patch_size, stride=patch_size
        )
        
        h = w = image_size // patch_size 

        seq_length = (image_size // patch_size) ** 2
        self.pos_embedding = nn.Parameter(torch.randn(1, seq_length, hidden_dim) * 0.02, requires_grad=True)
        self.cls_token = nn.Parameter(torch.randn(1, 1, hidden_dim) * 0.02, requires_grad=True)

        self.encoder = Encoder(
            num_layers,
            num_heads,
            hidden_dim,
            mlp_dim,
            dropout,
            attention_dropout,
            norm_layer,
        )
        
        self.seq_length = seq_length

        heads_layers: OrderedDict[str, nn.Module] = OrderedDict()
        if representation_size is None:
            heads_layers["head"] = nn.Linear(hidden_dim, num_classes)
        else:
            heads_layers["pre_logits"] = nn.Linear(hidden_dim, representation_size)
            heads_layers["act"] = nn.Tanh()
            heads_layers["head"] = nn.Linear(representation_size, num_classes)

        self.heads = nn.Sequential(heads_layers)
        
        # Init the patchify stem
        fan_in = self.conv_proj.in_channels * self.conv_proj.kernel_size[0] * self.conv_proj.kernel_size[1] // self.conv_proj.groups
        jax_lecun_normal(self.conv_proj, fan_in)

        if hasattr(self.heads, "pre_logits") and isinstance(self.heads.pre_logits, nn.Linear):
            fan_in = self.heads.pre_logits.in_features
            jax_lecun_normal(self.heads.pre_logits, fan_in)

        if isinstance(self.heads.head, nn.Linear):
            nn.init.zeros_(self.heads.head.weight)
            nn.init.zeros_(self.heads.head.bias)

    def _process_input(self, x: torch.Tensor) -> torch.Tensor:
        n, c, h, w = x.shape
        p = self.patch_size
        torch._assert(h == self.image_size, f"Wrong image height! Expected {self.image_size} but got {h}!")
        torch._assert(w == self.image_size, f"Wrong image width! Expected {self.image_size} but got {w}!")
        n_h = h // p
        n_w = w // p

        # (n, c, h, w) -> (n, hidden_dim, n_h, n_w)
        x = self.conv_proj(x)
        # (n, hidden_dim, n_h, n_w) -> (n, hidden_dim, (n_h * n_w))
        x = x.reshape(n, self.hidden_dim, n_h * n_w)

        # (n, hidden_dim, (n_h * n_w)) -> (n, (n_h * n_w), hidden_dim)
        # The self attention layer expects inputs in the format (N, S, E)
        # where S is the source sequence length, N is the batch size, E is the
        # embedding dimension
        x = x.permute(0, 2, 1)

        return x
    
    def forward(self, x: torch.Tensor):
        # Reshape and permute the input tensor
        x = self._process_input(x)
        n = x.shape[0]

        # Add positional embedding
        x = x + self.pos_embedding

        # Prepend the CLS token
        cls_token = self.cls_token.expand(n, -1, -1)
        x = torch.cat([cls_token, x], dim=1)

        # Pass through the encoder
        x = self.encoder(x)

        # Use only the CLS token for classification
        x = x[:, 0]

        # Pass through the classification head
        x = self.heads(x)

        return x

In [ ]:
def weight_decay_param(n, p):
    return p.ndim >= 2 and n.endswith('weight')

# create model
model = SimpleVisionTransformer(
    image_size=224,
    patch_size=16,
    num_layers=12,
    num_heads=6,
    hidden_dim=384,
    mlp_dim=1536,
)
model = nn.DataParallel(model)
model.to('cuda')

wd_params = [p for n, p in model.named_parameters() if weight_decay_param(n, p) and p.requires_grad]
non_wd_params = [p for n, p in model.named_parameters() if not weight_decay_param(n, p) and p.requires_grad]
original_model = model

weight_decay = 0.05
learning_rate = 1e-3

# Label smoothing loss
criterion = LabelSmoothing(smoothing=0.1)

optimizer = torch.optim.AdamW(
    [
        {"params": wd_params, "weight_decay": weight_decay},
        {"params": non_wd_params, "weight_decay": weight_decay},
    ],
    lr=learning_rate,
    betas=(0.9, 0.999)  # Set beta1=0.9 and beta2=0.999
)

warmup = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda step: step / warmup_try)
cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps - warmup_try)
scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, [warmup, cosine], [warmup_try])

In [ ]:
#Change_path_for_the_directory;This is the directory where model weights are to be saved
checkpoint_path = "/kaggle/working/"

def save_checkpoint(state, is_best, path, filename='imagenet_baseline_patchconvcheckpoint.pth.tar'):
    filename = os.path.join(path, filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(path, 'model_best.pth.tar'))

def save_checkpoint_step(step, model, best_acc1, optimizer, scheduler, checkpoint_path):
    # Define the filename with the current step
    filename = os.path.join(checkpoint_path, f'BaseLine_with_PE.pt')
    
    # Save the checkpoint
    torch.save({
        'step': step,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }, filename)
    

class Summary(Enum):
    NONE = 0
    AVERAGE = 1
    SUM = 2
    COUNT = 3

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f', summary_type=Summary.AVERAGE):
        self.name = name
        self.fmt = fmt
        self.summary_type = summary_type
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def all_reduce(self):
        if torch.cuda.is_available():
            device = torch.device("cuda")
        elif torch.backends.mps.is_available():
            device = torch.device("mps")
        else:
            device = torch.device("cpu")
        total = torch.tensor([self.sum, self.count], dtype=torch.float32, device=device)
        dist.all_reduce(total, dist.ReduceOp.SUM, async_op=False)
        self.sum, self.count = total.tolist()
        self.avg = self.sum / self.count
    
    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
    def summary(self):
        fmtstr = ''
        if self.summary_type is Summary.NONE:
            fmtstr = ''
        elif self.summary_type is Summary.AVERAGE:
            fmtstr = '{name} {avg:.3f}'
        elif self.summary_type is Summary.SUM:
            fmtstr = '{name} {sum:.3f}'
        elif self.summary_type is Summary.COUNT:
            fmtstr = '{name} {count:.3f}'
        else:
            raise ValueError('invalid summary type %r' % self.summary_type)
        
        return fmtstr.format(**self.__dict__)
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))
        
    def display_summary(self):
        entries = [" *"]
        entries += [meter.summary() for meter in self.meters]
        print(' '.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [ ]:
log_steps = 2500

wandb.login(key="40845bd710604d2ba6b7cdc034459067ea7ae3b8")

# Initialize a new run
wandb.init(project="ICLR_2025_Blog", name="BaseLine_with_PE")

In [ ]:
start_step = 0

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def validate(val_loader, model, criterion, step, use_wandb=False, print_freq=100):
    batch_time = AverageMeter('Time', ':6.3f', Summary.NONE)
    losses = AverageMeter('Loss', ':.4e', Summary.NONE)
    top1 = AverageMeter('Acc@1', ':6.2f', Summary.AVERAGE)
    top5 = AverageMeter('Acc@5', ':6.2f', Summary.AVERAGE)
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if torch.cuda.is_available():
                images = images.cuda(non_blocking=True)
                target = target.cuda(non_blocking=True)
            elif torch.backends.mps.is_available():
                images = images.to('mps')
                target = target.to('mps')

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                progress.display(i)

    progress.display_summary()
    
    if use_wandb:        
        log_data = {
            'val/loss': losses.avg,
            'val/acc@1': top1.avg,
            'val/acc@5': top5.avg,
        }
        wandb.log(log_data, step=step)

    return top1.avg

def train(train_loader, val_loader, start_step, total_steps, original_model, model, criterion, optimizer, scheduler, device):
    
    def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
        print(f"Loading checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        start_step = checkpoint['step']
        model.load_state_dict(checkpoint['state_dict'])
        best_acc1 = checkpoint['best_acc1']
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        print(f"Loaded checkpoint. Resuming from step {start_step}")
        return start_step, best_acc1
    
    # Load checkpoint
    start_step, best_acc1 = load_checkpoint("/kaggle/input/mmmmmmttttttt/BaseLine_with_PE.pt", original_model, optimizer, scheduler)
    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    print_freq = 100
    log_steps = 2500
    
    progress = ProgressMeter(
        total_steps,
        [batch_time, data_time, losses, top1, top5]
    )

    model.train()
    end = time.time()
    
    def infinite_loader():
        while True:
            yield from train_loader
            
    for step, (images, labels_a, labels_b, lam) in zip(range(start_step + 1, total_steps + 1), infinite_loader()):
        
        print(step)
        
        data_time.update(time.time() - end)

        images = images.to(device, non_blocking=True)
        labels_a = labels_a.to(device, non_blocking=True)
        labels_b = labels_b.to(device, non_blocking=True)
        
        # Convert lam to a tensor if it's not already one
        if not isinstance(lam, torch.Tensor):
            lam = torch.tensor(lam, device=device)
        else:
            lam = lam.to(device, non_blocking=True)

        output = model(images)
        loss = lam * criterion(output, labels_a) + (1 - lam) * criterion(output, labels_b)

        # Compute accuracy (this is an approximation for mixed labels)
        acc1_a, acc5_a = accuracy(output, labels_a, topk=(1, 5))
        acc1_b, acc5_b = accuracy(output, labels_b, topk=(1, 5))
        acc1 = lam * acc1_a + (1 - lam) * acc1_b
        acc5 = lam * acc5_a + (1 - lam) * acc5_b

        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0].item(), images.size(0))
        top5.update(acc5[0].item(), images.size(0))

        loss.backward()
        l2_grads = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

        batch_time.update(time.time() - end)
        end = time.time()
        
        if step % print_freq == 0:
            progress.display(step)
            if wandb:
                with torch.no_grad():
                    l2_params = sum(p.square().sum().item() for _, p in model.named_parameters())
                    
                samples_per_second_per_gpu = images.size(0) / batch_time.val
                samples_per_second = samples_per_second_per_gpu 
                log_data = {
                    "train/loss": losses.val,
                    'train/acc@1': top1.val,
                    'train/acc@5': top5.val,
                    "data_time": data_time.val,
                    "batch_time": batch_time.val,
                    "samples_per_second": samples_per_second,
                    "samples_per_second_per_gpu": samples_per_second_per_gpu,
                    "lr": scheduler.get_last_lr()[0],
                    "l2_grads": l2_grads.item(),
                    "l2_params": math.sqrt(l2_params)
                }
                wandb.log(log_data, step=step)
        
        if ((step % print_freq == 0) and ((step % log_steps != 0) and (step != total_steps))):        
            save_checkpoint_step(step, model, best_acc1, optimizer, scheduler, checkpoint_path)
                
        if step % log_steps == 0:
            acc1 = validate(val_loader, original_model, criterion, step)
            is_best = acc1 > best_acc1
            best_acc1 = max(acc1, best_acc1)
            
            save_checkpoint({
                'step': step,
                'state_dict': original_model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
                'scheduler' : scheduler.state_dict()
            }, is_best, checkpoint_path)
            
        elif step == total_steps:
            acc1 = validate(val_loader, original_model, criterion, step)
            is_best = acc1 > best_acc1
            best_acc1 = max(acc1, best_acc1)
            
            save_checkpoint({
                'step': step,
                'state_dict': original_model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
                'scheduler' : scheduler.state_dict()
            }, is_best, checkpoint_path)

        scheduler.step()

# Use the modified train function
train(train_loader, val_loader, start_step, total_steps, original_model, model, criterion, optimizer, scheduler, device)

In [ ]:
wandb.finish()

In [ ]:
# # Test dataset paths first
# print("=== Testing Dataset Paths ===")
# for path in train_dirs + val_dir:
#     print(f"Checking path {path}: {os.path.exists(path)}")
# print(f"Checking labels file {labels_file}: {os.path.exists(labels_file)}")

# # Test label loading
# print("\n=== Testing Label Loading ===")
# with open(labels_file, 'r') as f:
#     label_dict = json.load(f)
# print("Number of unique labels:", len(label_dict))
# print("First few labels:", list(label_dict.keys())[:5])

# # Test dataset creation
# print("\n=== Testing Dataset Creation ===")
# try:
#     test_dataset = ImageNet100Dataset(
#         root_dirs=train_dirs,
#         labels_file=labels_file,
#         transform=train_transform
#     )
#     print("Training dataset size:", len(test_dataset))
    
#     # Test single item loading
#     img, label = test_dataset[0]
#     print("Single image shape:", img.shape)
#     print("Single label:", label)
    
#     # Visualize transformations on one image
#     plt.figure(figsize=(15, 3))
    
#     # Original image
#     img_orig = Image.open(test_dataset.images[0]).convert('RGB')
#     plt.subplot(1, 4, 1)
#     plt.imshow(img_orig)
#     plt.title("Original")
    
#     # Apply transformations 3 times to show randomness
#     for i in range(3):
#         img_transformed, _ = test_dataset[0]
#         plt.subplot(1, 4, i+2)
#         plt.imshow(img_transformed.permute(1, 2, 0).clip(0, 1))
#         plt.title(f"Transform {i+1}")
    
#     plt.show()
    
# except Exception as e:
#     print("Error in dataset creation:", str(e))

# # Test dataloader
# print("\n=== Testing DataLoader ===")
# try:
#     test_loader = torch.utils.data.DataLoader(
#         test_dataset,
#         batch_size=batch_size,
#         shuffle=True,
#         collate_fn=collate_fn,
#         num_workers=2
#     )
    
#     # Get one batch
#     batch = next(iter(test_loader))
#     print("Batch length:", len(batch))  # Should be 4 (images, labels_a, labels_b, lam)
#     images, labels_a, labels_b, lam = batch
#     print("Batch shapes:")
#     print(f"Images: {images.shape}")
#     print(f"Labels A: {labels_a.shape}")
#     print(f"Labels B: {labels_b.shape}")
#     print(f"Lambda: {lam}")
    
# except Exception as e:
#     print("Error in dataloader:", str(e))
    
# print("=== Testing Model ===")

# # Test model creation
# try:
#     test_model = SimpleVisionTransformer(
#         image_size=256,
#         patch_size=16,
#         num_layers=12,
#         num_heads=6,
#         hidden_dim=384,
#         mlp_dim=1536,
#     )
#     print("Model created successfully")
    
#     # Print model summary
#     print("\nModel Architecture:")
#     print(test_model)
    
#     # Count parameters
#     total_params = sum(p.numel() for p in test_model.parameters())
#     trainable_params = sum(p.numel() for p in test_model.parameters() if p.requires_grad)
#     print(f"\nTotal parameters: {total_params:,}")
#     print(f"Trainable parameters: {trainable_params:,}")
    
#     # Test forward pass
#     print("\nTesting forward pass...")
#     test_model.to(device)
#     test_input = torch.randn(2, 3, 256, 256).to(device)  # batch size of 2
#     with torch.no_grad():
#         output = test_model(test_input)
#     print("Output shape:", output.shape)
    
#     # Test memory usage
#     if torch.cuda.is_available():
#         print("\nGPU Memory Usage:")
#         print(f"Allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
#         print(f"Cached: {torch.cuda.memory_reserved()/1024**2:.2f} MB")
    
# except Exception as e:
#     print("Error in model testing:", str(e))
    
# print("=== Testing Loss Function and Optimizer ===")

# try:
#     # Test loss function
#     print("\nTesting Label Smoothing Loss...")
#     test_criterion = LabelSmoothing(smoothing=0.1)
#     test_predictions = torch.randn(4, 100)  # 4 samples, 100 classes
#     test_targets = torch.tensor([0, 1, 2, 3])
#     test_loss = test_criterion(test_predictions, test_targets)
#     print("Test loss value:", test_loss.item())
    
#     # Test with different smoothing values
#     for smoothing in [0.0, 0.1, 0.2]:
#         criterion = LabelSmoothing(smoothing=smoothing)
#         loss = criterion(test_predictions, test_targets)
#         print(f"Loss with smoothing {smoothing}: {loss.item()}")
    
#     # Test optimizer
#     print("\nTesting Optimizer...")
#     test_model = SimpleVisionTransformer(
#         image_size=256,
#         patch_size=16,
#         num_layers=12,
#         num_heads=6,
#         hidden_dim=384,
#         mlp_dim=1536,
#     ).to(device)
    
#     # Test parameter grouping
#     wd_params = [p for n, p in test_model.named_parameters() if weight_decay_param(n, p) and p.requires_grad]
#     non_wd_params = [p for n, p in test_model.named_parameters() if not weight_decay_param(n, p) and p.requires_grad]
#     print(f"Parameters with weight decay: {len(wd_params)}")
#     print(f"Parameters without weight decay: {len(non_wd_params)}")
    
#     # Test optimizer creation
#     test_optimizer = torch.optim.AdamW(
#         [
#             {"params": wd_params, "weight_decay": weight_decay},
#             {"params": non_wd_params, "weight_decay": 0.0},
#         ],
#         lr=learning_rate,
#     )
#     print("Optimizer created successfully")
    
#     # Test scheduler
#     print("\nTesting Learning Rate Scheduler...")
#     test_warmup = torch.optim.lr_scheduler.LambdaLR(test_optimizer, lr_lambda=lambda step: step / warmup_try)
#     test_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(test_optimizer, T_max=total_steps - warmup_try)
#     test_scheduler = torch.optim.lr_scheduler.SequentialLR(test_optimizer, [test_warmup, test_cosine], [warmup_try])
    
#     # Print learning rates at different steps
#     steps_to_check = [0, warmup_try//2, warmup_try, total_steps//2, total_steps]
#     print("\nLearning rate at different steps:")
#     for step in steps_to_check:
#         for _ in range(step):
#             test_scheduler.step()
#         print(f"Step {step}: {test_scheduler.get_last_lr()[0]}")
    
# except Exception as e:
#     print("Error in loss/optimizer testing:", str(e))
    
# print("=== Testing Training Setup ===")

# # Test CUDA availability
# print("\nTesting CUDA Setup:")
# print(f"CUDA available: {torch.cuda.is_available()}")
# print(f"Current device: {device}")
# if torch.cuda.is_available():
#     print(f"GPU name: {torch.cuda.get_device_name(0)}")

# # Test Wandb initialization
# print("\nTesting Wandb Setup...")
# try:
#     wandb.init(project="ICLR_2025_Blog", name="test_run")
#     print("Wandb initialized successfully")
    
#     # Test logging
#     wandb.log({
#         "test_metric": 0.5,
#         "test_loss": 0.1
#     })
#     print("Wandb logging successful")
#     wandb.finish()
# except Exception as e:
#     print("Error in wandb setup:", str(e))

# # Test training loop components
# print("\nTesting Training Loop Components...")
# try:
#     # Test progress meter
#     batch_time = AverageMeter('Time', ':6.3f')
#     data_time = AverageMeter('Data', ':6.3f')
#     losses = AverageMeter('Loss', ':.4e')
#     top1 = AverageMeter('Acc@1', ':6.2f')
#     progress = ProgressMeter(
#         total_steps,
#         [batch_time, data_time, losses, top1],
#         prefix="Test: "
#     )
    
#     # Update meters
#     batch_time.update(0.5)
#     losses.update(0.1)
#     top1.update(95.0)
    
#     # Test display
#     progress.display(0)
    
#     # Test checkpoint saving
#     print("\nTesting Checkpoint Saving...")
#     dummy_state = {
#         'step': 0,
#         'state_dict': test_model.state_dict(),
#         'best_acc1': 0,
#         'optimizer': test_optimizer.state_dict(),
#         'scheduler': test_scheduler.state_dict()
#     }
#     save_checkpoint(dummy_state, False, checkpoint_path)
#     print(f"Checkpoint saved to {checkpoint_path}")
    
# except Exception as e:
#     print("Error in training loop components:", str(e))

# print("\nAll tests completed!")